# Семинар 10: продвинутые функции

Давайте напишем функцию, которая возвращает факториал от заданного числа.

> Напоминаю, что $n! = n×(n-1)×(n-2)×...×1$



In [ ]:
# ваш код сюда

А теперь по-другому.



In [9]:
def factorial_recursive(n: int) -> int:
    if n == 1:
        return n

    else:
        return n * factorial_recursive(n-1)

In [10]:
num = 7
print(f"{num}! это {factorial_recursive(num)}")

7! это 5040


> Рекурсивная функция $-$ это та, которая вызывает сама себя.

1. Благодаря условной конструкции переменная $n$ вернется только в том случае, если ее значение будет равно $1$. Это еще называют *условием завершения*. Рекурсия останавливается в момент удовлетворения условиям.
2. Любой алгоритм, реализованный в рекурсивной форме, может быть переписан в итерационном виде и наоборот. Остаеся вопрос, надо ли это, и насколько это будет это эффективно.
3. Как это **работает**?

        3 * (3-1) * ((3-1)-1)  
        так как 3-1-1 равно 1, рекурсия остановилась

        /\ factorial_recursive(1) - последний вызов
        || factorial_recursive(2) - второй вызов
        || factorial_recursive(3) - первый вызов

Так генерируется *стек*. Это происходит благодаря процессу **LIFO** (last in, first out, «последним пришел — первым ушел»). Первые вызовы функции "не знают ответа", поэтому они добавляются в стек.

4. Как это **выводится**?

Как только в стек добавляется вызов `factorial_recursive(1)`, для которого ответ имеется, стек начинает "разворачиваться" в обратном порядке, выполняя все вычисления с реальными значениями.

        factorial_recursive(1) завершается, отправляет 1 в
        factorial_recursive(2) и выпадает из стека.
        factorial_recursive(2) завершается, отправляет 2*1 в
        factorial_recursive(3) и выпадает из стека. Инструкция else здесь завершается, возвращается 3 * 2 = 6, и из стека выпадает последний слой.

Проще всего проверить это и проиллюстрировать на примере с ошибкой:

In [20]:
def recursive_error(n: int):
    if n == 0:
        raise RuntimeError("Just checking call stack")

    recursive_error(n - 1)

In [22]:
recursive_error(4)

RuntimeError: Just checking call stack

### Задача №1

Напишите рекурсивную функцию `sum_squares()` для вычисления суммы $n$ первых членов: $$1 + \frac 1 4 + \frac 1 9 + ... + \frac 1 n$$
    
```python
>>> print(sum_squares(9))
<<< 1.5397677311665408
```

In [ ]:
# ваш код тут

5. Наиболее распространенная ошибкa, связанная с рекурсией $-$ *бесконечная рекурсия*, когда цепочка вызовов функций никогда не завершается и продолжается, пока не кончится свободная память в компьютере.

![img](https://media.geeksforgeeks.org/wp-content/uploads/20200707093844/WhatsApp-Image-2020-07-07-at-9.37.31-AM.jpeg)

```python
# бесконечная рекурсия ин э натшелл

def short_story():
    print("У попа была собака, он ее любил.")
    print("Она съела кусок мяса, он ее убил,")
    print("И в землю закопал и надпись написал:")
    short_story()
```

Две наиболее распространенные причины для бесконечной рекурсии:

* Неправильное оформление выхода из рекурсии.<br>Например, если мы в программе вычисления факториала забудем поставить проверку `if n == 1`, то `factorial_recursive(1)` вызовет `factorial_recursive(0)`, тот вызовет `factorial_recursive(-1)` и т. д.
* Рекурсивный вызов с неправильными параметрами.<br>Например, если функция `factorial_recursive(n)` будет вызывать `factorial_recursive(n)`, то также получится бесконечная цепочка.

## Проблема лимита рекурсии

Пусть мы хотим посчитать факториал для 999:

In [39]:
num = 999
print(f"{num}! это {factorial_recursive(num)}")  # упс...

RecursionError: maximum recursion depth exceeded

In [41]:
import sys


sys.getrecursionlimit()

1000

6. **NB!**<br>Рекурсия в Python дефолтно имеет ограничение в 1000 слоев, если не... (см. ниже).

In [50]:
sys.setrecursionlimit(10 ** 6)  # надо быть аккуратными

factorial_recursive(1000)

4023872600770937735437024339230039857193748642107146325437999104299385123986290205920442084869694048004799886101971960586316668729948085589013238296699445909974245040870737599188236277271887325197795059509952761208749754624970436014182780946464962910563938874378864873371191810458257836478499770124766328898359557354325131853239584630755574091142624174743493475534286465766116677973966688202912073791438537195882498081268678383745597317461360853795345242215865932019280908782973084313928444032812315586110369768013573042161687476096758713483120254785893207671691324484262361314125087802080002616831510273418279777047846358681701643650241536913982812648102130927612448963599287051149649754199093422215668325720808213331861168115536158365469840467089756029009505376164758477284218896796462449451607653534081989013854424879849599533191017233555566021394503997362807501378376153071277619268490343526252000158885351473316117021039681759215109077880193931781141945452572238655414610628921879602238389714760

## Проблема отсутствия мемоизации

Пусть мы хотим посчитать $n$-ое число Фибоначчи

$F_1 = F_2 = 1$

$F_n = F_{n - 1} + F_{n - 2}$

In [51]:
def fib(n: int) -> int:
    if n <= 2:
        return 1
    return fib(n - 1) + fib(n - 2)  # рекурсивно берем предыдущие числа Фибоначчи

А теперь давайте подумаем: сколько будет работать такой код?

In [52]:
%%time
fib(8)

CPU times: user 9 µs, sys: 1e+03 ns, total: 10 µs
Wall time: 16.9 µs


21

In [53]:
%%time
fib(34)  # тут уже будет долго

CPU times: user 1.11 s, sys: 3.63 ms, total: 1.11 s
Wall time: 1.15 s


5702887

In [54]:
%%time
fib(40)  # и чем дальше, тем хуже, а 40 - это же еще не то чтобы много

CPU times: user 14.4 s, sys: 23.9 ms, total: 14.5 s
Wall time: 14.6 s


102334155

7. Рекурсивные функции обычно занимают **больше места** в памяти по сравнению с итеративными из-за постоянного добавления новых слоев в стек в памяти, хоть их производительность и выше.

Вся проблема в том, что у нас есть дубликаты вызовов. Поэтому питон делает много дополнительных действий. Чем дальше по номеру число Фибоначчи, тем больше придется решать одинаковых абсолютно вызовов рекурсии.

![](https://upload.wikimedia.org/wikipedia/commons/5/5f/FibbonacciRecurisive.png)

Можно решить данную проблему мемоизацией:

In [59]:
_fib_cache = {}


def mem_fib(n: int) -> int:
    if n <= 2:
        _fib_cache[n] = 1
    result = _fib_cache.get(n)
    if result is None:
        result = mem_fib(n - 2) + mem_fib(n - 1)
        _fib_cache[n] = result
    return result

In [67]:
%%time
mem_fib(1000)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.11 µs


43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875

Очевидно, что это уже получается не очень удобно, поэтому кэширование в питоне есть в том числе и встроенное.

Есть так называемые декораторы `@cache` и `@lru_cache` (least recently used). Второй выкидывает из кэша элементы, когда его размер достиг заданного максимума. При этом `@cache` устроен вот так:

```python
def cache(user_function, /):
    return lru_cache(maxsize=None)(user_function)
```

У `@lru_cache` параметр maxsize по умолчанию равен 128, можно увеличить при желании.

In [70]:
from functools import lru_cache

# @lru_cache(maxsize=128)
@lru_cache(maxsize=128)
def fib(n: int) -> int:
    if n <= 2:
        return 1

    return fib(n - 1) + fib(n - 2)

In [71]:
%%time
fib(1000)

CPU times: user 0 ns, sys: 838 µs, total: 838 µs
Wall time: 845 µs


43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875

### Задача №2

Дан список чисел, необходимо просуммировать его при помощи рекурсивной функции `summary()`.
    
```python
>>> print(summary([2, 4, 5, 6, 7]))
<<< 24
```
    

In [ ]:
# место для вашего кода

### Задача №3

Напишите функцию `flatten()`, которая принимает на вход список, состоящий из других списков, и возвращает обычный список, в котором присутствуют все элементы из вложенных списков. Эта операция производится при помощи рекурсии.
    
```python
>>> print(flatten([[1, 2], [3, 4], 5, [6, [7]]]))
<<< [1, 2, 3, 4, 5, 6, 7]
```

In [ ]:
# ваш код тутб